In [1]:
import pandas as pd

df = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/arquivomesclado.csv')

df_filled = df.fillna(method='ffill').fillna(method='bfill')

df_filled = df_filled.apply(lambda x: x.fillna(x.mean()) if x.dtype.kind in 'biufc' else x)

df_filled.to_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv', index=False)



In [3]:
import pandas as pd

df_dolar = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv')
df_ibovespa = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/ibovespa.csv')
df_price_coffee = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/price_coffee.csv')
df_selic = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/selic.csv')
df_weather = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/weather.csv')

dfs = [df_dolar, df_ibovespa, df_price_coffee, df_selic, df_weather]
for df in dfs:
    df['Date'] = pd.to_datetime(df['Date'])

dfs_filtered = [df[df['Date'].isin(df_price_coffee['Date'])] for df in dfs]

df_merged = df_price_coffee
for df in dfs_filtered:
    df_merged = df_merged.merge(df, on='Date', how='left')

df_merged.to_csv(('C:/Users/milen/OneDrive/Documentos/TCC/Bases/arquivomesclado.csv'), index=False)


In [6]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
import pandas as pd
import numpy as np

file_path = 'C:/Users/milen/OneDrive/Documentos/TCC/new/final.csv'
data = pd.read_csv(file_path)

data['Data'] = pd.to_datetime(data['Data'])
data['Preco_Dolar'] = data['Preco_Dolar'].str.replace(',', '.').astype(float)

z_scores = np.abs(stats.zscore(data['Preco_Dolar']))
threshold = 3
data['Preco_Dolar_No_Outliers'] = data['Preco_Dolar'].where(z_scores < threshold, np.nan)

data['Preco_Dolar_No_Outliers'] = data['Preco_Dolar_No_Outliers'].interpolate()

X = data[['Preco_Dolar_No_Outliers']]  
y = data['Preco_Dolar'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=365, shuffle=False)

rf_model = RandomForestRegressor(n_estimators=100, random_state=0)
rf_model.fit(X_train, y_train)

rf_forecast = rf_model.predict(X_test)

mse_rf = mean_squared_error(y_test, rf_forecast)
r2_rf = r2_score(y_test, rf_forecast)

print("MSE:", mse_rf)
print("R2 Score:", r2_rf)

result_data_rf = pd.DataFrame({
    'Date': data['Data'].iloc[-365:],
    'Real Value': y_test.values,
    'Forecast Value': rf_forecast
})

print(result_data_rf.reset_index(drop=True))


MSE: 0.006944562328766365
R2 Score: 0.9999934676028719
          Date  Real Value  Forecast Value
0   2022-05-30      268.04        268.2440
1   2022-05-31      267.96        268.2440
2   2022-06-01      275.51        275.5040
3   2022-06-02      277.12        276.9530
4   2022-06-03      270.35        270.3637
..         ...         ...             ...
360 2023-11-06      178.19        178.1976
361 2023-11-07      178.62        178.6228
362 2023-11-08      181.48        181.4768
363 2023-11-09      181.93        181.9296
364 2023-11-10      180.91        180.8922

[365 rows x 3 columns]
